In [1]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time
import random
import nest_asyncio
from tqdm.notebook import tqdm

In [2]:
# 경제 관련 키워드 리스트



# nest_asyncio를 적용하여 중첩된 이벤트 루프 허용
nest_asyncio.apply()


In [3]:
class NewsCrawler:
    def __init__(self, start_date, end_date, page_num):
        self.start_date = start_date
        self.end_date = end_date
        self.page_num = page_num
        self.data = []
        self.total_days = (end_date - start_date).days + 1
        
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        self.economic_keywords = [
            # 일반 경제 용어
            '경제', '경제성장', '경제위기', '금융', '물가', '인플레이션', '디플레이션', '경기', '불황', '호황',

            # 금융 시장 관련
            '주식', '증시', '코스피', '코스닥', '환율', '금리', '채권', '펀드', '선물', '옵션', '파생상품', '외환',

            # 무역 및 국제 경제
            '무역', '수출', '수입', '관세', 'FTA', '세계경제', '글로벌경제', '미중무역전쟁', '보호무역', '자유무역',

            # 고용 및 노동
            '고용', '실업', '일자리', '노동', '임금',

            # 부동산
            '부동산', '주택', '아파트','재개발',

            # 경제 지표
            'GDP', '무역수지', '경상수지', '소비자물가지수', '생산자물가지수', '실업률', '경제성장률', '기준금리',
        ]
        self.economic_keywords_dict = {
                "일반 경제 용어": [
                    '경제', '경제성장', '경제위기', '금융', '물가', '인플레이션', '디플레이션', '경기', '불황', '호황'
                ],
                "금융 시장 관련": [
                    '주식', '증시', '코스피', '코스닥', '환율', '금리', '채권', '펀드', '선물', '옵션', '파생상품', '외환'
                ],
                "무역 및 국제 경제": [
                    '무역', '수출', '수입', '관세', 'FTA', '세계경제', '글로벌경제', '미중무역전쟁', '보호무역', '자유무역'
                ],
                "고용 및 노동": [
                    '고용', '실업', '일자리', '노동', '임금'
                ],
                "부동산": [
                    '부동산', '주택', '아파트', '재개발'
                ],
                "경제 지표": [
                    'GDP', '무역수지', '경상수지', '소비자물가지수', '생산자물가지수', '실업률', '경제성장률', '기준금리'
                ],
                "비트 코인": [
                    '비트코인', 'BTC', ''
                ]

            }

        
    async def get_news_urls(self, session, date):
        urls = []
        for page in range(1, self.page_num + 1):
            url = f'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101&date={date}&page={page}'
            async with session.get(url, headers=self.headers) as response:
                html = await response.text()
                soup = BeautifulSoup(html, 'html.parser')

                news_list = soup.select('.newsflash_body .type06_headline li dl')
                news_list.extend(soup.select('.newsflash_body .type06 li dl'))

                for news in news_list:
                    news_url = news.a['href']
                    urls.append(news_url)

        return urls
    
    def is_keward_news(self,title,content, key):
        title = title or ""
        content = content or ""
        text = f"{title} {content}"
        keywards = self.economic_keywords_dict[key]
        matching_keywords = [keyword for keyword in keywards if keyword in text]
        return (bool(matching_keywords), matching_keywords)
        

    def is_economic_news(self,title, content):
        title = title or ""
        content = content or ""
        text = f"{title} {content}"
        matching_keywords = [keyword for keyword in self.economic_keywords if keyword in text]
        return (bool(matching_keywords), matching_keywords)


    async def crawl(self):
        async with aiohttp.ClientSession() as session:
            pbar = tqdm(total=self.total_days, desc="Crawling Progress")
            current_date = self.start_date
            while current_date <= self.end_date:
                await self.crawl_date(session, current_date)
                current_date += timedelta(days=1)
                pbar.update(1)
                await asyncio.sleep(random.uniform(1, 3))
            pbar.close()

    async def get_news_content(self, session, url):
        async with session.get(url, headers=self.headers) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')

            try:
                article = soup.select_one('#dic_area')
                if article:
                    title = article.select_one('strong.media_end_summary')
                    title = title.text if title else "제목 없음"

                    # 이미지 설명 제거
                    for img_desc in article.select('em.img_desc'):
                        img_desc.decompose()

                    content = ' '.join(p.strip() for p in article.strings if p.strip())

                    return title, content
                else:
                    return None, None
            except:
                return None, None

    async def process_url(self, session, url, date, keyward='일반 경제 용어'):
        title, content = await self.get_news_content(session, url)
        
        
        if keyward == None:
            re = self.is_economic_news(title, content)
        else:
            # print(title, content)
            re = self.is_keward_news(title,content,keyward)

        if re[0]:  # Only add if it's economic news
            self.data.append({
                'date': date,
                'title': title,
                'content': content,
                'url': url,
                'keyword': re[1]
            })

    async def crawl_date(self, session, date, keyward):
        date_str = date.strftime('%Y%m%d')
        urls = await self.get_news_urls(session, date_str)
        tasks = [self.process_url(session, url, date_str, keyward) for url in urls]
        print(tasks)
        await asyncio.gather(*tasks)

    async def crawl(self, keyward):
        async with aiohttp.ClientSession() as session:
            pbar = tqdm(total=self.total_days, desc="Crawling Progress")
            current_date = self.start_date
            while current_date <= self.end_date:
                await self.crawl_date(session, current_date, keyward)
                current_date += timedelta(days=1)
                pbar.update(1)
                await asyncio.sleep(random.uniform(1, 3))  # 요청 간 랜덤 지연
            pbar.close()

    def run(self, keyward = '일반 경제 용어'):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.crawl(keyward))
        return pd.DataFrame(self.data)



In [4]:
start_date = datetime(2025, 4, 22)
end_date = datetime(2025, 4, 23)
page_num = 10  # 각 날짜마다 크롤링할 페이지 수

crawler = NewsCrawler(start_date, end_date, page_num)
crawler.economic_keywords_dict['일반 경제 용어']
df = crawler.run(keyward = '부동산')
df

Crawling Progress:   0%|          | 0/2 [00:00<?, ?it/s]

[<coroutine object NewsCrawler.process_url at 0x7f18dd2d7be0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d45e0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d44c0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d43a0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4280>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4160>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4040>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4b80>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4ca0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4dc0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d4ee0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5000>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5120>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5240>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5360>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5480>, <corout

,date,title,content,url,keyword
0,20250422,4월 세계경제전망 발표멕시코·태국 이어 낙폭 3위 최우선 협상 5개국 일제히 하락...,4월 세계경제전망 발표 멕시코·태국 이어 낙폭 3위 최우선 협상 5개국 일제히 하락...,https://n.news.naver.com/mnews/article/005/000...,[부동산]
1,20250422,제목 없음,공정거래위원회가 KB국민은행·우리은행·신한은행·하나은행 등 국내 4대 은행의 주택담...,https://n.news.naver.com/mnews/article/422/000...,[주택]
2,20250422,제목 없음,[서울=뉴시스]이재준 기자 = 중국 온라인 여행사 퉁청뤼싱(同程旅行)이 부동산 개발...,https://n.news.naver.com/mnews/article/003/001...,[부동산]
3,20250422,제목 없음,[앵커] 조기 대선 정국을 맞으며 세종시 집값이 들썩이고 있습니다. 새로 들어설 정...,https://n.news.naver.com/mnews/article/422/000...,"[부동산, 주택, 아파트]"
4,20250422,제목 없음,지자체별 맞춤형 만남 행사 잇따라 결혼땐 1천만원 등 장려금 경쟁도 “지방자치단체가...,https://n.news.naver.com/mnews/article/009/000...,"[주택, 아파트]"
5,20250422,제목 없음,[이데일리 최정훈 기자] 이스라엘이 가자지구의 주택과 난민 캠프 등을 공습해 최소 ...,https://n.news.naver.com/mnews/article/018/000...,[주택]
6,20250422,美 스탠퍼드대 연구 결과집단주의 강한 韓 문화 영향,美 스탠퍼드대 연구 결과 집단주의 강한 韓 문화 영향 게티이미지뱅크 한국인은 재택근...,https://n.news.naver.com/mnews/article/021/000...,[주택]
7,20250422,제목 없음,"[KBS 부산] [앵커] 조기 대선을 앞두고 부산 부동산 시장은 침체 속, 눈치 보...",https://n.news.naver.com/mnews/article/056/001...,"[부동산, 아파트, 재개발]"
8,20250422,제목 없음,"[KBS 부산] [앵커] 부산 원도심, 중구의 인구가 4만 명 밑으로 떨어졌습니다....",https://n.news.naver.com/mnews/article/056/001...,"[주택, 재개발]"
9,20250422,제목 없음,[KBS 춘천] [앵커] 평창군의 인구가 4만 명 선마저 무너질 위기에 놓였습니다...,https://n.news.naver.com/mnews/article/056/001...,[주택]


In [5]:
for i in df['url']:
    print(i)

https://n.news.naver.com/mnews/article/005/0001771679?sid=101
https://n.news.naver.com/mnews/article/422/0000733650?sid=101
https://n.news.naver.com/mnews/article/003/0013199733?sid=101
https://n.news.naver.com/mnews/article/422/0000733636?sid=101
https://n.news.naver.com/mnews/article/009/0005480866?sid=101
https://n.news.naver.com/mnews/article/018/0005994649?sid=101
https://n.news.naver.com/mnews/article/021/0002704901?sid=101
https://n.news.naver.com/mnews/article/056/0011937103?sid=101
https://n.news.naver.com/mnews/article/056/0011937102?sid=101
https://n.news.naver.com/mnews/article/056/0011937088?sid=101
https://n.news.naver.com/mnews/article/056/0011937100?sid=101
https://n.news.naver.com/mnews/article/024/0000096567?sid=101
https://n.news.naver.com/mnews/article/024/0000096566?sid=101
https://n.news.naver.com/mnews/article/082/0001322506?sid=101
https://n.news.naver.com/mnews/article/024/0000096565?sid=101
https://n.news.naver.com/mnews/article/087/0001112385?sid=101
https://

In [6]:
# 크롤링 실행
start_date = datetime(2025, 4, 21)
end_date = datetime(2025, 4, 23)
page_num = 10  # 각 날짜마다 크롤링할 페이지 수

crawler = NewsCrawler(start_date, end_date, page_num)
df = crawler.run(keyward = '일반 경제 용어')

# CSV로 저장
df.to_csv(f'naver_economic_news_{start_date}_{end_date}_async.csv', index=False, encoding='utf-8-sig')
print("크롤링 완료 및 CSV 저장 완료")

Crawling Progress:   0%|          | 0/3 [00:00<?, ?it/s]

[<coroutine object NewsCrawler.process_url at 0x7f18dd2d6c20>, <coroutine object NewsCrawler.process_url at 0x7f18dd2d6e60>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5ea0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5fc0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5d80>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5c60>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5b40>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5a20>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5900>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d5120>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d57e0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d56c0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d60e0>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d6200>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d6320>, <coroutine object NewsCrawler.process_url at 0x7f18dd1d6440>, <corout

In [7]:
df

,date,title,content,url,keyword
0,20250421,제목 없음,도널드 트럼프 미국 대통령이 제롬 파월 미국 연방준비제도 의장을 '중대 실패자'로 ...,https://n.news.naver.com/mnews/article/422/000...,"[경제, 경기]"
1,20250421,주가 변동성 크지 않은 통신·금융주 위주 투자트럼프 취임 이후 4달간 4~5%대 수...,주가 변동성 크지 않은 통신·금융주 위주 투자 트럼프 취임 이후 4달간 4~5%대 ...,https://n.news.naver.com/mnews/article/005/000...,[금융]
2,20250421,현행 최저임금 1만30원 ‘높다’ 47.4%,현행 최저임금 1만30원 ‘높다’ 47.4% 게티이미지뱅크 상시근로자 300인 미만...,https://n.news.naver.com/mnews/article/021/000...,[물가]
3,20250421,제목 없음,[KBS 전주]오영주 중소벤처기업부 장관이 오늘(21) 전주시 원도심 상권을 방문해...,https://n.news.naver.com/mnews/article/056/001...,[경제]
4,20250421,"SNS서 ""사실상 인플레 없다""관세發 경기 침체 우려에 금리 인하 압박파월에 ""미스...","SNS서 ""사실상 인플레 없다"" 관세發 경기 침체 우려에 금리 인하 압박 파월에 ""...",https://n.news.naver.com/mnews/article/277/000...,"[경제, 인플레이션, 경기]"
...,...,...,...,...,...
293,20250423,증선위 “형식적 MOU 과장 홍보” 김 여사·이종호 대표는 포함 안돼,증선위 “형식적 MOU 과장 홍보” 김 여사·이종호 대표는 포함 안돼 금융 당국이 ...,https://n.news.naver.com/mnews/article/005/000...,[금융]
294,20250423,제목 없음,[KBS 광주] [앵커] '2026 세계섬박람회' 개막이 오늘로 5백일 남았습니다....,https://n.news.naver.com/mnews/article/056/001...,[경제]
295,20250423,인터넷기업협회 '인터넷산업규제 백서' 발간,인터넷기업협회 '인터넷산업규제 백서' 발간 한국인터넷기업협회는 23일 '인터넷산업규...,https://n.news.naver.com/mnews/article/469/000...,[경제]
296,20250423,제목 없음,미국과 일본이 이르면 이번주 중반 2차 관세협상에 돌입할 것으로 관측된다. 23일 ...,https://n.news.naver.com/mnews/article/277/000...,[경제]
